In [31]:
def bring_task_to_close_type(matrix_cost, ai_vector_storage, bj_vector_shop):
    sum_ai = sum(ai_vector_storage)
    sum_bj = sum(bj_vector_shop)

    if sum_ai < sum_bj:
        ai_vector_storage.append(sum_bj - sum_ai)
        matrix_cost.append([0] * len(bj_vector_shop))
    elif sum_ai > sum_bj:
        bj_vector_shop.append(sum_ai - sum_bj)
        matrix_cost = [row + [0] for row in matrix_cost]

    return matrix_cost, ai_vector_storage, bj_vector_shop

In [32]:
def find_base_solution_min_cost(matrix_cost, ai_vector_storage, bj_vector_shop):
    supply = ai_vector_storage.copy()
    demand = bj_vector_shop.copy()
    
    num_suppliers = len(supply)
    num_consumers = len(demand)
    
    result = [[None] * num_consumers for _ in range(num_suppliers)]
    
    i, j = 0, 0
    
    while i < num_suppliers and j < num_consumers:
        min_val = min(supply[i], demand[j])
            
        if min_val > 0: 
            result[i][j] = min_val
        
        supply[i] -= min_val
        demand[j] -= min_val
        
        if supply[i] == 0 and i < num_suppliers:
            i += 1
        if demand[j] == 0 and j < num_consumers:
            j += 1
    
    return result

In [33]:
def print_matrix_x_ij(matrix_x_ij):
    print("Матрица перевозок (x_ij):")
    for row in matrix_x_ij:
        print("\t ".join(str(x) if x is not None else "0" for x in row))

In [34]:
def method_potentials(matrix_cost, matrix_x_ij):
    rows = len(matrix_cost)
    cols = len(matrix_cost[0])

    ai_potential = [None] * rows
    bj_potential = [None] * cols

    ai_potential[0] = 0
    record_ai_potentials, record_bj_potentials = 1, 0

    while record_ai_potentials != len(ai_potential) or record_bj_potentials != len(bj_potential):
        for i in range(rows):
            for j in range(cols):
                if matrix_x_ij[i][j] is not None:
                    if ai_potential[i] is None and bj_potential[j] is not None:
                        ai_potential[i] = matrix_cost[i][j] - bj_potential[j]
                        record_ai_potentials += 1
                    if ai_potential[i] is not None and bj_potential[j] is None:
                        bj_potential[j] = matrix_cost[i][j] - ai_potential[i]
                        record_bj_potentials += 1

    return ai_potential, bj_potential

In [35]:
def get_first_non_basic_or_optimal(matrix_x_ij, ai_potential, bj_potential, matrix_cost):
    for i in range(len(matrix_x_ij)):
        for j in range(len(matrix_x_ij[i])):
            if matrix_x_ij[i][j] is None and ai_potential[i] + bj_potential[j] > matrix_cost[i][j]:
                return i, j
    return True

In [36]:
def find_closed_loop(matrix_x_ij, i_start, j_start):
    stack = [((i_start, j_start), None, [])]
    visited = set()

    while stack:
        current, direction, path = stack.pop()
        if current == (i_start, j_start) and len(path) >= 4:
            return path

        if current in visited and current != (i_start, j_start):
            continue

        path.append(current)
        visited.add(current)

        neighbors = []
        if direction is None:
            for col in range(len(matrix_x_ij[0])):
                if matrix_x_ij[current[0]][col] is not None:
                    neighbors.append(((current[0], col), "col"))
            for row in range(len(matrix_x_ij)):
                if matrix_x_ij[row][current[1]] is not None:
                    neighbors.append(((row, current[1]), "row"))
        if direction == "row":
            for col in range(len(matrix_x_ij[0])):
                if (matrix_x_ij[current[0]][col] is not None or
                        (current[0] == i_start and col == j_start and len(path) > 3)):
                    neighbors.append(((current[0], col), "col"))
        if direction == "col":
            for row in range(len(matrix_x_ij)):
                if (matrix_x_ij[row][current[1]] is not None or
                        (row == i_start and current[1] == j_start and len(path) > 3)):
                    neighbors.append(((row, current[1]), "row"))

        for neighbor, next_direction in neighbors:
            if neighbor not in visited or neighbor == (i_start, j_start):
                stack.append((neighbor, next_direction, path.copy()))

    return None

In [37]:
def recalculate_cycle(matrix_x_ij, path):
    minimal = float('inf')
    for row, col in path[1::2]:
        current_value = matrix_x_ij[row][col]
        if current_value is not None and current_value > 0:
            minimal = min(minimal, current_value)

    for row, col in path[0::2]:
        if matrix_x_ij[row][col] is None:
            matrix_x_ij[row][col] = 0
        matrix_x_ij[row][col] += minimal

    for row, col in path[1::2]:
        matrix_x_ij[row][col] -= minimal
        if matrix_x_ij[row][col] == 0:
            matrix_x_ij[row][col] = None

In [38]:
def calculate_optimal_value_and_print_matrixs(matrix_cost, matrix_x_ij):
    L_star = 0
    for i in range(len(matrix_cost)):
        for j in range(len(matrix_cost[i])):
            if matrix_x_ij[i][j] is not None:
                L_star += matrix_cost[i][j] * matrix_x_ij[i][j]
    print_matrix_x_ij(matrix_x_ij)
    print(f'\nЗначение функции L = {L_star}')

In [39]:
def solve_transport_task(matrix_cost, ai_vector_storage, bj_vector_shop):
    matrix_cost, ai_vector_storage, bj_vector_shop = bring_task_to_close_type(matrix_cost, ai_vector_storage, bj_vector_shop)
    matrix_x_ij = find_base_solution_min_cost(matrix_cost, ai_vector_storage, bj_vector_shop)
    
    while True:
        ai_potential, bj_potential = method_potentials(matrix_cost, matrix_x_ij)
        result = get_first_non_basic_or_optimal(matrix_x_ij, ai_potential, bj_potential, matrix_cost)

        if result is True:
            calculate_optimal_value_and_print_matrixs(matrix_cost, matrix_x_ij)
            break
        else:
            i_start, j_start = result
            path = find_closed_loop(matrix_x_ij, i_start, j_start)
            if path is None:
                raise KeyError("Не удалось найти замкнутый цикл")
            recalculate_cycle(matrix_x_ij, path)

In [40]:
matrix_cost = [[40, 36, 9, 20], [26, 11, 22, 26], [6, 3, 12, 3], [5, 37, 33, 26]]
a_i = [24, 42, 23, 36]
b_j = [35, 29, 21, 35]

solve_transport_task(matrix_cost, a_i, b_j)

Матрица перевозок (x_ij):
0	 0	 21	 3	 0
0	 29	 0	 9	 4
0	 0	 0	 23	 0
35	 0	 0	 0	 1

Значение функции L = 1046
